In [15]:
import json
with open('twitter.json') as file:
    df = json.load(file)

In [32]:
data = df['tweets']

In [10]:
import string
import nltk
def tokenize_split(article):
    punct = set(string.punctuation)
    collection = article.split()
    token = [ ''.join( c for c in w if c not in punct ) for w in collection ]
    token = [ w.lower() for w in token ]
    return token

# Stopwords removal.
nltk.download('stopwords')
from nltk.corpus import stopwords

def removeStopWords(token):
    stopWords = set(stopwords.words('english'))
    return [ c for c in token if c not in stopWords ]

# Lemmatization
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def lemmatize(concisedToken):
    return [ WordNetLemmatizer().lemmatize(c) for c in concisedToken ]

# Stemming
from nltk.stem import PorterStemmer

def stemming(concisedToken):
    return [ PorterStemmer().stem(c) for c in concisedToken]

def preprocess(article):
    token = tokenize_split(article)
    concisedToken = removeStopWords(token)
    stems = [ c for c in stemming(concisedToken) if c != '']
    lemmas = [ c for c in lemmatize(concisedToken) if c != '']
    return stems, lemmas

[nltk_data] Downloading package stopwords to /Users/yhkuo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/yhkuo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
###sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

def sentimentAnalysis(headline):
    return SIA().polarity_scores(headline)

In [29]:
from datetime import datetime

def convertUnixTime(timestamp):
    return datetime.utcfromtimestamp(timestamp).strftime('%m/%d/%Y') #'%H:%M:%S'
    

In [33]:
processed_data = {}
for tweet in data:
    processed_tweet = {}
    text = tweet['text']
    timestamp = tweet['timestamp']
    tweet_id = tweet['tweet_id']
    
    sentiment = sentimentAnalysis(text)
    
    processed_tweet['tweet_id'] = tweet_id
    processed_tweet['text'] = text
    processed_tweet['sentiment'] = sentiment
    
    date = convertUnixTime(timestamp)
    if date in processed_data:
        processed_data[date].append(processed_tweet)
    else:
        processed_data[date] = [processed_tweet]



NameError: name 'OrderedDict' is not defined

In [34]:
from collections import OrderedDict

result = dict(OrderedDict(sorted(processed_data.items(), key=lambda t: t[0])))
with open('processed_tweet.json', 'w') as file:
    json.dump(result, file)